###### Importing Libraries we need

In [2]:
import json
import csv
import requests
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.stats import linregress
from IPython.display import clear_output

##### went to https://crashviewer.nhtsa.dot.gov/CrashAPI and got the list of states and built a dictionary off of it

In [3]:
state_id = {1:["Alabama","AL"],
            2:["Alaska","AK"],
            4:["Arizona","AZ"],
            5:["Arkansas","AR"],
            6:["California","CA"],
            8:["Colorado","CO"],
            9:["Conneticut","CT"],
            10:["Delaware", "DE"],
            11:["District of Columbia","DC"],
            12:["Florida","FL"],
            13:["Georgia", "GA"],
            15:["Hawaii","HI"],
            16:["Idaho","ID"],
            17:["Illinois","IL"],
            18:["Indiana","IN"],
            19:["Iowa", "IA"],
            20:["Kansas", "KS"],
            21:["Kentucky","KY"],
            22:["Louisiana","LA"],
            23:["Maine","ME"],
            24:["Maryland","MD"],
            25:["Massachusetts","MA"],
            26:["Michigan","MI"],
            27:["Minnesota","MN"],
            28:["Mississippi","MS"],
            29:["Missouri","MO"],
            30:["Montana","MT"],
            31:["Nebraska","NB"],
            32:["Nevada","NV"],
            33:["New Hampshire","NH"],
            34:["New Jersey","NJ"],
            35:["New Mexico","NM"],
            36:["New York","NY"],
            37:["North Carolina","NC"],
            38:["North Dakota","ND"],
            39:["Ohio","OH"],
            40:["Oklahoma","OK"],
            41:["Oregon","OR"],
            42:["Pennsylvania","PA"],
            43:"Puerto Rico",
            44:["Rhode Island","RI"],
            45:["South Carolina","SC"],
            46:["South Dakota","SD"],
            47:["Tennessee","TN"],
            48:["Texas","TX"],
            49:["Utah","UT"],
            50:["Vermont","VT"],
            51:["Virginia","VA"],
            52:"Virgin Islands",
            53:["Washington", "WA"],
            54:["West Virginia","WV"],
            55:["Wisconsin","WI"],
            56:["Wyoming","WY"]}



##### Create a while loop for user input

In [ ]:
state_list=["6","8","12","17","36","39","42","48"]
state_name_list = ["California","Colorado", "Florida", "Illinois", "New York", "Ohio","Pennsylvania","Texas"]
inquiry = str
while inquiry != "done" or inquiry != "":
    print(f"By default there is\n {state_list} \n {state_name_list}")
    
    inquiry = input("Please input the state you are interested in getting Data for by Name or code (ex : CO for Colorado). if you are done type done")
   
    if inquiry == "done":
                    break
    else:
                    for i in state_id:
                        if inquiry == state_id[i][0] or inquiry == state_id[i][1]:
                            state_num = i
                            state_list.append(state_num)
                            state_name_list.append(inquiry)
                            print(f"{inquiry} added")
                            clear_output()
                    
print(f"You added {state_name_list} to the list")

By default there is
 ['6', '8', '12', '17', '36', '39', '42', '48', 41] 
 ['California', 'Colorado', 'Florida', 'Illinois', 'New York', 'Ohio', 'Pennsylvania', 'Texas', 'Oregon']


In [ ]:
# collect query results for selected states --> result is series of dictionaries, I think?

FromYear = "2008"
ToYear = "2018"
Results2 = []
for State in state_list:
    query_url = url + "fromCaseYear=" + FromYear + "&toCaseYear=" + ToYear + "&state=" + State + "&county=" + "&format=json"
    response = requests.get(query_url).json()
    Results2.append(response["Results"][0])
    print(State)
    pprint(Results2)

In [ ]:
#should use a couple of for loops here but could do later

# assign results to each state
California = Results2[0]
print(California)
Colorado = Results2[1]
Florida = Results2[2]
Illinois = Results2[3]
New_York = Results2[4]
Ohio = Results2[5]
Pennsylvania = Results2[6]
Texas = Results2[7]

# make df from each series

cali_df = pd.DataFrame(California)
colo_df = pd.DataFrame(Colorado)
flor_df = pd.DataFrame(Florida)
illi_df = pd.DataFrame(Illinois)
NY_df = pd.DataFrame(New_York)
ohio_df = pd.DataFrame(Ohio)
penn_df = pd.DataFrame(Pennsylvania)
tex_df = pd.DataFrame(Texas)

# set column heading so can merge
cali_df['State'] = "California"
colo_df['State'] = "Colorado"
flor_df['State'] = "Florida"
illi_df['State'] = "Illinois"
NY_df['State'] = "New York"
ohio_df['State'] = "Ohio"
penn_df['State'] = "Pennsyvlania"
tex_df['State'] = "Texas"

# merge all dataframes to one df of results from all states
merged = cali_df.merge(flor_df, left_on=['State','CaseYear','CrashCounts','TotalFatalCounts'], right_on=['State','CaseYear','CrashCounts','TotalFatalCounts'],how="outer")
merged = merged.merge(illi_df, left_on=['State','CaseYear','CrashCounts','TotalFatalCounts'], right_on=['State','CaseYear','CrashCounts','TotalFatalCounts'],how="outer")
merged = merged.merge(NY_df, left_on=['State','CaseYear','CrashCounts','TotalFatalCounts'], right_on=['State','CaseYear','CrashCounts','TotalFatalCounts'],how="outer")
merged = merged.merge(ohio_df, left_on=['State','CaseYear','CrashCounts','TotalFatalCounts'], right_on=['State','CaseYear','CrashCounts','TotalFatalCounts'],how="outer")
merged = merged.merge(penn_df, left_on=['State','CaseYear','CrashCounts','TotalFatalCounts'], right_on=['State','CaseYear','CrashCounts','TotalFatalCounts'],how="outer")
merged = merged.merge(tex_df, left_on=['State','CaseYear','CrashCounts','TotalFatalCounts'], right_on=['State','CaseYear','CrashCounts','TotalFatalCounts'],how="outer")
merged = merged.merge(colo_df, left_on=['State','CaseYear','CrashCounts','TotalFatalCounts'], right_on=['State','CaseYear','CrashCounts','TotalFatalCounts'],how="outer")

merged

In [ ]:
# convert TotalFatalCounts to integer
merged['TotalFatalCounts'] = merged['TotalFatalCounts'].astype(int)

In [ ]:
# group df by states
grouped_states = merged.groupby('State')
grouped_states.head()

In [ ]:
# plot total fatal crashes by state over time
fig = go.Figure()
for states,group in grouped_states['State']:
    year = merged.loc[merged["State"] == states]["CaseYear"]
    crashes = merged.loc[merged["State"] == states]["TotalFatalCounts"]

    # Add traces
    fig.add_trace(go.Scatter(x=year, y=crashes,
                             mode='lines',
                             name=states))
fig.show()

In [ ]:
# import MAU data for mobile app adoption, reindex
MAU_data = pd.read_csv('MAU_data.csv')
MAU_data = MAU_data.fillna(0)
MAU_data_index = MAU_data.set_index('App')
MAU_data_index

In [ ]:
# drop years from crash data to ones overlapping with MAU data
grouped_years = merged.groupby('CaseYear')
crashes_per_year = grouped_years['TotalFatalCounts'].agg('sum')
#crashes_per_year_trimmed = crashes_per_year.drop(['2018','2017','2016','2015'])
crashes_per_year

Out[11]:

CaseYear

In [ ]:
# drop years from MAU_data to match years in crash data
MAU_data_trimmed = MAU_data_index.drop(['2002','2003','2004','2005','2006','2007','2008','2009','2019'],axis=1)

# transpose data
MAU_trim_trans = MAU_data_trimmed.transpose()
MAU_trim_total = MAU_trim_trans['Total']

# set type to float
MAU_trim_total.astype('float').dtypes
MAU_trim_total

In [ ]:
# set x and y values for scatter plot
x_values = MAU_trim_total
y_values = crashes_per_year
print(x_values)
print(y_values)

In [ ]:
# scatter plot of MAUs of mobile apps vs. Fatal Crashes for 5 years (2010-2014)
plt.scatter(x_values,y_values)
plt.xlabel('MAUs')
plt.ylabel('Fatal Crashes')
plt.title('Fatal Crashes in 8 States vs. MAUs of Top Mobile Apps in the US, 2010-2018')
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.annotate(line_eq,(3000000000,15500),fontsize=15,color="red")
plt.plot(x_values,regress_values,"r-")
plt.show()
plt.show()

In [ ]:
# import and read smartphone adoption datafile
phone_adoption_df = pd.read_csv('smartphone adoption.csv')

#trim years to match crash data years
phone_adoption_trim = phone_adoption_df.drop([9,10,11,12,13])

In [ ]:
# plot scatterplot and regression line of smartphone users and crash fatalities
x_values2 = phone_adoption_trim['number of smartphone users in US (millions)']
y_values = crashes_per_year
plt.scatter(x_values2,y_values)
plt.title('Fatal Crashes in 8 States vs. Number of Smartphone Users in US, 2010-2018')
plt.xlabel('number of smartphone users in US')
plt.ylabel('Fatal Crashes')
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values2, y_values)
regress_values = x_values2 * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.annotate(line_eq,(75,15500),fontsize=15,color="red")
plt.plot(x_values2,regress_values,"r-")
plt.show()